<h1>Code Snippets for Parsing the Gigantic Stack Overflow Dataset (D3)</h1>

<p><b>Step1:</b> <u>Download the data dump from the official Stack Exchange <a href='https://archive.org/download/stackexchange'>archive</a>. </u>It contains 8 XML files namely stackoverflow.com-Badges, stackoverflow.com-Comments, stackoverflow.com-PostHistory, stackoverflow.com-PostLinks, stackoverflow.com-Posts, stackoverflow.com-Tags, stackoverflow.com-Users and stackoverflow.com-Votes</p> 

<p><b>Step 2: </b><u>Split the large sized XML files to smaller chunks</u>. Apart from stackoverflow.com-PostLinks, and stackoverflow.com-Tags, all other files are too large to process easily in one go. So the next step is to divide them into small chunks. For this purpose, <a href='https://git-scm.com/download/win'>git bash</a> can be used which provides split command to split files line by line or by size. We used the split by line option. After extracting each of the above mentioned compressed files, open git bash terminal in the respective direcotry and type the following commands respectively. </p><br>
<ul>split -l 50000 -d -a 5 Badges.xml Badges</ul>
<ul>split -l 50000 -d -a 5 Users.xml Users</ul>
<ul>split -l 50000 -d -a 8 Comments.xml Comments</ul>
<ul>split -l 50000 -d -a 5 Votes.xml Votes</ul>
<ul>split -l 50000 -d -a 10 Posts.xml Posts</ul>
<ul>split -l 50000 -d -a 10 PostHistory.xml PostHistory</ul>


<p><b>Step 3: </b><u>Converting splitted files to proper XML files.</u> As the split operation was performed line by line, the first chunk contains XML header whereas the last chunk contains the closing tag of the respective file. The rest of the chunks lack proper XML header/footer. Make a directory and move all the respective chunks generated by running each of the commands in that directory e.g. if the original <code>Post.xml</code> was placed in <code>E:\Dataset_2024\stackoverflow.com-Posts</code>, after running the above command (<code>split -l 50000 -d -a 10 Posts.xml Posts</code>, the same folder will contain more than 1100 files starting from <code>Posts0000000000</code>. Excluding the original large sized Posts.xml, move the rest of the files (small chunks) to some other directory e.g. <code>PostsSplit</code>. Provide the path to this direcotry as <code>input_path</code>. Similarly, make another directory to store the resultant small sized XML files and provide its path in <code>output_path</code>. We used the following code snippet to convert these chunks into small sized XML files. Replace the XML header/footer strings in the code snippet (e.g <code> posts </code> and <code> /posts </code>) based on the file that you are dealing with e.g. posts, users, votes and so on.</p>

In [ ]:
import glob
import os
input_path =r'E:\Dataset_2024\stackoverflow.com-PostHistory\PostHistorySplit'
output_path=r'E:\Dataset_2024\stackoverflow.com-PostHistory\PostHistorySplitXML\\'

for filename in glob.glob(os.path.join(input_path, '*')):
    with open(os.path.join(os.getcwd(), filename), 'r',encoding='utf8') as f: 
        print("Reading file {}".format(filename))
        Lines = f.readlines()
        xml_lines=[]
        xml_lines.append("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<posthistory>")
        xml_lines.extend(Lines)
        xml_lines.append("</posthistory>")

        head, tail = os.path.split(filename)

        outputfile = output_path+tail+'.xml'      
        print("Writing file to {}".format(outputfile))
        with open(outputfile, "w",encoding='utf8') as output:
            output.write(''.join (xml_lines))

<p><b>Step4: </b><u>Manually correcting the first and last XML file</u>. The above code snippet does not consider any special cases. Hence, after executing the above code snippet the first file will contain the header twice and the last file will contain the closing tag twice. We need to remove the redundant tags from the first and last file manually to make them valid.</p>

<p><b>Step 5: </b><u>Saving XML based data in database for further querying.</u> We used MS SQL Server Developer Edition to store the data as the Express Edition restricts the database size to 10GB at maximum. However, in our case that dataset was much larger than that. It is worth mentioning here that the Developer Edition cannot be used in production environment. We further used SQL Server Management Studio (SSMS) to easily create and manipulate databases using a GUI. Both <a href='https://www.microsoft.com/en-us/sql-server/sql-server-downloads?msockid=236f7f3cd0c269f836b36be8d4c26fdb'>SQL Server </a> and <a href='https://learn.microsoft.com/en-us/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-ver16'> SSMS </a> are available for use freely by Microsoft. The default location used by SQL Server to store database files on a Windows based system is <code>C:\Program Files\Microsoft SQL Server\MSSQL16.MSSQLSERVER\MSSQL\DATA\</code>. Using <code>New Database</code> wizard in SSMS, create new database and change the default location for storing the dataabase files to some other directory as your <code>C:</code> drive may have limited space. Instead of storing all data in a single database, we created separate databases corresponding to each of the above mentioned XML files (i.e. <code> StackOverflowBadgesDb_April2024, StackOverflowCommentsDb_April2024, StackOverflowPostHistoryDb_April2024, StackOverflowPostLinksDb_April2024, StackOverflowPostsDb_April2024, StackOverflowTagsDb_April2024, StackOverflowUsersDb_April2024, StackOverflowVotesDb_April2024 </code> respectively.)</p>

<p><b>Step 6: </b><u>Creating Tables for Storing Data</u>. After creating the respective databases, the next step is to create tables in which the actual data will be stored from the XML files. We used the following SQL queries to create corresponding tables. </p>


<p><b>Step 7: </b><u>Reading XML files and inserting records in respective database tables</u>. Now we need to read the small sized XML files one by one and store the data in the respective tables created above. For this purpose, the code snippet used for parsing each of the 8 XML file types namely Badges, Comments, PostHistory, PostLinks, Posts, Tags, Users, and Votes is given below. Before using these snippets, replace the server and database names and the path of directory containing splitted XML files. </p>

<h2>Badges Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os

server = 'DESKTOP-DEK23E9'
database = 'StackOverflowBadgesDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

badges_xml_path ='E:\\Dataset_2024\\stackoverflow.com-Badges\\BadgesSplitXML'

for filename in glob.glob(os.path.join(badges_xml_path, '*')):

    print(filename)
    
    badges_df = pd.read_xml(filename)
    badges_df = badges_df.fillna(0)

    print (badges_df.head())
    print('*************')
 
    query = "insert into StackOverflowBadges (Id, UserId, Name, Date, Class,TagBased) values (?,?,?,?,?,?)"

    cursor.executemany(query, badges_df.values.tolist())
    cursor.commit()
    print("Added in database")
    

<h2>Comments Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
server = 'DESKTOP-DEK23E9'
database = 'StackOverflowCommentsDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

comments_xml_path ='E:\\Dataset_2024\\stackoverflow.com-Comments\\CommentsSplitXML'

for filename in glob.glob(os.path.join(comments_xml_path, '*')):

    print(filename)
    comments_df = pd.read_xml(filename)
    comments_df = comments_df.fillna(0)

    print (comments_df.head())
    print('*************')
    columnnames= list(comments_df.columns.values)
    columns=','.join(columnnames)
    query="insert into StackOverflowComments ("+columns+") values (?,?,?,?,?,?,?)"

    cursor.executemany(query, comments_df.values.tolist())
    cursor.commit()
    print("Added in database")

<h2>PostHistory Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os



server = 'DESKTOP-DEK23E9'
database = 'StackOverflowPostHistoryDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
post_history_xml_path ='E:\\Dataset_2024\\stackoverflow.com-PostHistory\\PostHistorySplitXML\\'

for filename in glob.glob(os.path.join(post_history_xml_path, '*')):

    print(filename)
    PostHistory_df = pd.read_xml(filename)
    
    PostHistory_df =  PostHistory_df.fillna(value={'Id':0,'PostHistoryTypeId':0,'PostId':0,'RevisionGUID':0,'CreationDate':0,'UserId':0,'Text':'','ContentLicense':'','UserDisplayName':'', 'Comment':''})
    print (PostHistory_df.head())
    print('*************')
    columnnames= list(PostHistory_df.columns.values)
    columns=','.join(columnnames)
    query="insert into StackOverflowPostHistory ("+columns+") values (?,?,?,?,?,?,?,?,?,?)"
    
    # query = "insert into StackOverflowPostHistory(Id,PostHistoryTypeId,PostId,RevisionGUID,CreationDate,UserId,Text,ContentLicense,Comment,UserDisplayName) values (?,?,?,?,?,?,?,?,?,?)"

    cursor.executemany(query, PostHistory_df.values.tolist())
    cursor.commit()
    print("Added in database")

<h2>PostLinks Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc

server = 'DESKTOP-DEK23E9'
database = 'StackOverflowPostLinksDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

PostLinks_xml_path ='E:\Dataset_2024\stackoverflow.com-PostLinks\PostLinks.xml'
PostLinks_df = pd.read_xml(PostLinks_xml_path)
PostLinks_df = PostLinks_df.fillna(0)

print (PostLinks_df.head())
print('*************')
 
query = "insert into StackOverflowPostLinks (Id,CreationDate,PostId,RelatedPostId,LinkTypeId) values (?,?,?,?,?)"

cursor.executemany(query, PostLinks_df.values.tolist())
cursor.commit()
print("Added in database")

<h2>Posts Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
import re
from datetime import datetime
import traceback

server = 'DESKTOP-DEK23E9'
database = 'StackOverflowPostsDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
posts_xml_path ='E:\\Dataset_2024\\stackoverflow.com-Posts\\PostsSplitXML'

for filename in glob.glob(os.path.join(posts_xml_path, '*')):

    print(filename)
    posts_df = pd.read_xml(filename)
    print(posts_df.head())
    posts_df =  posts_df.fillna(value={'Id':0,'PostTypeId':0,'AcceptedAnswerId':0,'CreationDate':'','Score':0,'ViewCount':0,'OwnerUserId':0, 'LastEditorUserId':0, 'LastEditDate':0, 'LastActivityDate':0,'AnswerCount':0,'CommentCount':0, 'ParentId':0, 'CommunityOwnedDate':0 ,  'ClosedDate':0,
                            'FavoriteCount':0})
    posts_df['Id']=posts_df['Id'].astype(int)
    posts_df['PostTypeId']=posts_df['PostTypeId'].astype(int)
    posts_df['AcceptedAnswerId']=posts_df['AcceptedAnswerId'].astype(int)
    posts_df['CreationDate']= pd.to_datetime(posts_df['CreationDate'], format="mixed")
    posts_df['Score']=posts_df['Score'].astype(int)
    posts_df['ViewCount']=posts_df['ViewCount'].astype(int)
    posts_df['OwnerUserId']=posts_df['OwnerUserId'].astype(int)
    posts_df['LastEditorUserId']=posts_df['LastEditorUserId'].astype(int)
    posts_df['LastEditDate']= pd.to_datetime(posts_df['LastEditDate'], format="mixed")
    posts_df['LastActivityDate']= pd.to_datetime(posts_df['LastActivityDate'], format="mixed")
    posts_df['AnswerCount']=posts_df['AnswerCount'].astype(int)
    posts_df['CommentCount']=posts_df['CommentCount'].astype(int)
    posts_df['ParentId']=posts_df['ParentId'].astype(int)
    posts_df['CommunityOwnedDate']= pd.to_datetime(posts_df['CommunityOwnedDate'], format="mixed")
    posts_df['ClosedDate']= pd.to_datetime(posts_df['ClosedDate'], format="mixed")
    columnnames= list(posts_df.columns.values)
    columns=','.join(columnnames)
    if 'FavoriteCount' in posts_df.columns:
        posts_df['FavoriteCount']=posts_df['FavoriteCount'].astype(int)
        
        query="insert into StackOverflowPosts ("+columns+") values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    else:
        query="insert into StackOverflowPosts ("+columns+") values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    
    #     print(posts_df.columns)

    posts_df = posts_df.fillna('')
    
    #     query = "insert into StackOverflowPosts (Id, PostTypeId, CreationDate, Score, ViewCount, Body,OwnerDisplayName, LastActivityDate, Title, Tags, AnswerCount,CommentCount, FavoriteCount, ContentLicense, ParentId,OwnerUserId, LastEditorUserId, LastEditorDisplayName,LastEditDate, AcceptedAnswerId, CommunityOwnedDate, ClosedDate, Tag0,Tag1,Tag2, Tag3, Tag4 ) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
#     query = "insert into StackOverflowPosts (Id, PostTypeId, AcceptedAnswerId, CreationDate, Score,ViewCount, Body, OwnerUserId, LastEditorUserId, LastEditDate, LastActivityDate, Title, Tags, AnswerCount, CommentCount,ContentLicense, ParentId,CommunityOwnedDate, ClosedDate, OwnerDisplayName, LastEditorDisplayName, FavoriteCount, Tag0,Tag1,Tag2, Tag3, Tag4,VaderRawPositiveScore, VaderRawNegativeScore, VaderRawNeutralScore, VaderRawCompoundScore,VaderRawCompoundSentiment ) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

    cursor.executemany(query, posts_df.values.tolist())
    cursor.commit()
    print("Added in database")

<h2>Tags Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc

server = 'DESKTOP-DEK23E9'
database = 'StackOverflowTagsDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

row_list = []
errorcount = 0
tags_xml_path ='E:\\Dataset_2024\\stackoverflow.com-Tags\\Tags.xml'
tags_df = pd.read_xml(tags_xml_path)
tags_df = tags_df.fillna('')

print (tags_df.head())
print('*************')

query = "insert into StackOverflowTags (Id, TagName, TagCount ,ExcerptPostId ,WikiPostId) values (?,?,?,?,?)"

cursor.executemany(query, tags_df.values.tolist())
cursor.commit()
print("Added in database")

<h2>Users Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os

server = 'DESKTOP-DEK23E9'
database = 'StackOverflowUsersDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

users_xml_path ='E:\\Dataset_2024\\stackoverflow.com-Users\\UsersSplitXML'

for filename in glob.glob(os.path.join(users_xml_path, '*')):
    print(filename)
    users_df = pd.read_xml(filename)
    users_df = users_df.fillna(0)
    
    print (users_df.head())
    print(users_df.columns)
    print('*************')
    columnnames= list(users_df.columns.values)
    columns=','.join(columnnames)
#     print(columnnames)
    query="insert into StackOverflowUsers ("+columns+") values (?,?,?,?,?,?,?,?,?,?,?,?)"
#     query = "insert into StackOverflowUsers ( Id, Reputation, CreationDate, DisplayName, LastAccessDate, Views, UpVotes, DownVotes, AccountId, WebsiteUrl, AboutMe,ProfileImageUrl, Location) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

    cursor.executemany(query, users_df.values.tolist())
    cursor.commit()
    print("Added in database")
    

<h2>Votes Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os

server = 'DESKTOP-DEK23E9'
database = 'StackOverflowVotesDb_April2024' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

votes_xml_path ='E:\\Dataset_2024\\stackoverflow.com-Votes\\VotesSplitXML'

for filename in glob.glob(os.path.join(votes_xml_path, '*')):

    print(filename)
    votes_df = pd.read_xml(filename)
    votes_df = votes_df.fillna(0)

    print (votes_df.head())
    print('*************')
    columnnames= list(votes_df.columns.values)
    columns=','.join(columnnames)
    query="insert into StackOverflowVotes ("+columns+") values (?,?,?,?,?,?)"
    # query = "insert into StackOverflowVotes (Id, PostId,VoteTypeId,CreationDate) values (?,?,?,?)"
    # query = "insert into StackOverflowVotes (Id, PostId,VoteTypeId,CreationDate,BountyAmount) values (?,?,?,?,?)"
#     query = "insert into StackOverflowVotes (Id, PostId,VoteTypeId,CreationDate,UserId,BountyAmount) values (?,?,?,?,?,?)"

    cursor.executemany(query, votes_df.values.tolist())
    cursor.commit()
    print("Added in database")